# COLAS MM1

In [15]:
import numpy as np

In [16]:
class Customer:
    def __init__(self, arrival_time):
        self.arrival_time = arrival_time
        self.start_service_time = None
        self.end_service_time = None

    def start_service(self, start_time):
        self.start_service_time = start_time

    def end_service(self, end_time):
        self.end_service_time = end_time

    def time_in_system(self):
        if self.end_service_time is not None:
            return self.end_service_time - self.arrival_time
        return None


In [17]:
class Server:
    def __init__(self):
        self.current_customer = None
        self.busy = False

    def start_service(self, customer, current_time):
        self.current_customer = customer
        self.busy = True
        self.current_customer.start_service(current_time)

    def end_service(self, current_time):
        if self.current_customer:
            self.current_customer.end_service(current_time)
            self.busy = False
            finished_customer = self.current_customer
            self.current_customer = None
            return finished_customer
        return None

    def is_busy(self):
        return self.busy

In [18]:
class Simulation:
    def __init__(self, lambda_rate, mu_rate, time_end):
        self.lambda_rate = lambda_rate
        self.mu_rate = mu_rate
        self.time_end = time_end
        self.server = Server()
        self.customers = []
        self.time = 0
        self.total_time_in_system = 0
        self.total_customers_served = 0
        self.total_time_in_queue = 0  # Total time customers spend waiting in the queue
        self.queue_length_time_product = 0  # To integrate queue length over time

    def generate_next_arrival(self):
        return np.random.exponential(1.0 / self.lambda_rate)

    def generate_service_time(self):
        return np.random.exponential(1.0 / self.mu_rate)

    def run(self):
        next_arrival_time = self.time + self.generate_next_arrival()
        last_event_time = self.time  # Track time of the last event for queue length integration

        while self.time < self.time_end:
            next_event_time = next_arrival_time

            if self.server.is_busy() and self.server.current_customer.end_service_time < next_arrival_time:
                next_event_time = self.server.current_customer.end_service_time

            # Update queue length time product before changing the time
            self.queue_length_time_product += len(self.customers) * (next_event_time - last_event_time)
            last_event_time = next_event_time

            self.time = next_event_time

            if self.server.is_busy() and self.time >= self.server.current_customer.end_service_time:
                finished_customer = self.server.end_service(self.time)
                if finished_customer:
                    self.total_time_in_system += finished_customer.time_in_system()
                    self.total_customers_served += 1
                    self.total_time_in_queue += finished_customer.start_service_time - finished_customer.arrival_time

            if self.time >= next_arrival_time:
                new_customer = Customer(next_arrival_time)
                if not self.server.is_busy():
                    self.server.start_service(new_customer, self.time)
                    new_customer.end_service(self.time + self.generate_service_time())
                else:
                    self.customers.append(new_customer)
                next_arrival_time = self.time + self.generate_next_arrival()

            if not self.server.is_busy() and len(self.customers) > 0:
                next_customer = self.customers.pop(0)
                self.server.start_service(next_customer, self.time)
                next_customer.end_service(self.time + self.generate_service_time())

        NS = self.total_time_in_system / self.time_end
        TS = self.total_time_in_system / self.total_customers_served if self.total_customers_served > 0 else 0
        NQ = self.queue_length_time_product / self.time_end
        TQ = self.total_time_in_queue / self.total_customers_served if self.total_customers_served > 0 else 0

        return NS, TS, NQ, TQ


In [19]:
# Adjusted simulation duration for demonstration
lambda_rate = 2  # arrival rate per time unit
mu_rate = 3  # service rate per time unit
time_end = 50000  # simulation time

sim = Simulation(lambda_rate, mu_rate, time_end)
NS, TS, NQ, TQ = sim.run()

print(f"Average number of users in the system (NS): {NS}")
print(f"Average time spent by users in the system (TS): {TS}")
print(f"Average number of users in the queue (NQ): {NQ}")
print(f"Average time spent by users in the queue (TQ): {TQ}")


Average number of users in the system (NS): 2.034609060333361
Average time spent by users in the system (TS): 1.0131909748089563
Average number of users in the queue (NQ): 1.3650680468947765
Average time spent by users in the queue (TQ): 0.6797525004024374


# COLAS MM1K

In [20]:
class Customer:
    def __init__(self, arrival_time):
        self.arrival_time = arrival_time
        self.start_service_time = None
        self.end_service_time = None

    def start_service(self, start_time):
        self.start_service_time = start_time

    def end_service(self, end_time):
        self.end_service_time = end_time

    def time_in_system(self):
        if self.end_service_time is not None:
            return self.end_service_time - self.arrival_time
        return None

In [21]:
class Server:
    def __init__(self):
        self.current_customer = None
        self.busy = False

    def start_service(self, customer, current_time):
        self.current_customer = customer
        self.busy = True
        self.current_customer.start_service(current_time)

    def end_service(self, current_time):
        if self.current_customer:
            self.current_customer.end_service(current_time)
            self.busy = False
            finished_customer = self.current_customer
            self.current_customer = None
            return finished_customer
        return None

    def is_busy(self):
        return self.busy

In [22]:
class SimulationMM1K:
    def __init__(self, lambda_rate, mu_rate, time_end, K):
        self.lambda_rate = lambda_rate
        self.mu_rate = mu_rate
        self.time_end = time_end
        self.K = K
        self.server = Server()
        self.customers = []
        self.time = 0
        self.total_time_in_system = 0
        self.total_time_in_queue = 0  # Total time customers spend waiting in the queue
        self.total_customers_served = 0
        self.lost_customers = 0
        self.queue_length_time_product = 0  # To integrate queue length over time

    def generate_next_arrival(self):
        return np.random.exponential(1.0 / self.lambda_rate)

    def generate_service_time(self):
        return np.random.exponential(1.0 / self.mu_rate)

    def run(self):
        next_arrival_time = self.time + self.generate_next_arrival()
        last_event_time = self.time  # Track time of the last event for queue length integration

        while self.time < self.time_end:
            next_event_time = next_arrival_time

            if self.server.is_busy() and self.server.current_customer.end_service_time < next_arrival_time:
                next_event_time = self.server.current_customer.end_service_time

            # Update queue length time product before changing the time
            self.queue_length_time_product += len(self.customers) * (next_event_time - last_event_time)
            last_event_time = next_event_time

            self.time = next_event_time

            if self.server.is_busy() and self.time >= self.server.current_customer.end_service_time:
                finished_customer = self.server.end_service(self.time)
                if finished_customer:
                    self.total_time_in_system += finished_customer.time_in_system()
                    self.total_customers_served += 1
                    self.total_time_in_queue += finished_customer.start_service_time - finished_customer.arrival_time

            if self.time >= next_arrival_time:
                if not self.server.is_busy() or len(self.customers) < self.K - 1:
                    new_customer = Customer(next_arrival_time)
                    if not self.server.is_busy():
                        self.server.start_service(new_customer, self.time)
                        new_customer.end_service(self.time + self.generate_service_time())
                    else:
                        self.customers.append(new_customer)
                else:
                    self.lost_customers += 1
                next_arrival_time = self.time + self.generate_next_arrival()

            if not self.server.is_busy() and len(self.customers) > 0:
                next_customer = self.customers.pop(0)
                self.server.start_service(next_customer, self.time)
                next_customer.end_service(self.time + self.generate_service_time())

        NS = self.total_time_in_system / self.time_end
        TS = self.total_time_in_system / self.total_customers_served if self.total_customers_served > 0 else 0
        NQ = self.queue_length_time_product / self.time_end
        TQ = self.total_time_in_queue / self.total_customers_served if self.total_customers_served > 0 else 0

        return NS, TS, NQ, TQ, self.lost_customers


In [23]:
# Example usage
lambda_rate = 2  # arrival rate per time unit
mu_rate = 3  # service rate per time unit
time_end = 50000  # simulation time
K = 5  # System capacity including the server

sim = SimulationMM1K(lambda_rate, mu_rate, time_end, K)
NS, TS, NQ, TQ, lost_customers = sim.run()

print(f"Average number of users in the system (NS): {NS}")
print(f"Average time spent by users in the system (TS): {TS}")
print(f"Average number of users in the queue (NQ): {NQ}")
print(f"Average time spent by users in the queue (TQ): {TQ}")
print(f"Number of lost customers: {lost_customers}")

Average number of users in the system (NS): 1.4339760262824066
Average time spent by users in the system (TS): 0.7499011757446353
Average number of users in the queue (NQ): 0.7972919520453489
Average time spent by users in the queue (TQ): 0.41694416606868817
Number of lost customers: 4847


# Forma matematica

In [24]:
# Definimos los parámetros dados
lambda_rate = 2
mu_rate = 3
K = 5

# Calculamos P0 usando la fórmula proporcionada
P0 = (1 - lambda_rate/mu_rate) / (1 - (lambda_rate/mu_rate)**(K+1))

# Calculamos NS para el sistema M/M/1/K/∞
NS_MMK = lambda_rate * mu_rate * (1 - (lambda_rate/mu_rate)**K * (K*(1 - lambda_rate/mu_rate) + 1)) / (mu_rate * (mu_rate - lambda_rate) * (1 - (lambda_rate/mu_rate)**(K+1)))
NS_MMI = lambda_rate / (mu_rate - lambda_rate)

print("P0:", P0)
print("NS_MMK:", NS_MMK)
print("TS_MMK:", NS_MMK / lambda_rate)
print("Tq_MMK:", NS_MMK / lambda_rate - 1 / mu_rate)
print("Nq_MMK:", (NS_MMK / lambda_rate - 1/mu_rate) * lambda_rate)
print("NS_MMI:", NS_MMI)
print("TS_MMI:", NS_MMI / lambda_rate)
print("Tq_MMI:", NS_MMI / lambda_rate - 1 / mu_rate)
print("Nq_MMI:", (NS_MMI / lambda_rate - 1/mu_rate) * lambda_rate)

P0: 0.3654135338345865
NS_MMK: 1.422556390977444
TS_MMK: 0.711278195488722
Tq_MMK: 0.37794486215538864
Nq_MMK: 0.7558897243107773
NS_MMI: 2.0
TS_MMI: 1.0
Tq_MMI: 0.6666666666666667
Nq_MMI: 1.3333333333333335
